# Reviewing model versions

## Introduction

This cookbook will guide you through the handling and comparison of an existing computational model in jinko.  
In particular, you will be able to retrieve two specified model versions and output an html report containing the differences  


Linked resources: [Jinko](https://jinko.ai/project/e0fbb5bb-8929-439a-bad6-9e12d19d9ae4?labels=59a37e3f-471f-4958-a519-aa9ee128f5f7).

In [ ]:
# Jinko specifics imports & initialization
# Please fold this section and do not change
import jinko_helpers as jinko

# Connect to Jinko (see README.md for more options)
jinko.initialize()

In [ ]:
import zipfile
import io
from typing import List, Any
import json
from deepdiff import DeepDiff
import difflib
import re
from IPython.display import HTML

Firstly, it might be interesting to see which models are in a particular folder and which versions are available in the first place for a particular model. We can see an example for how to query this information from the API in the next cell.

In [ ]:
# List all models in a particular folder
folderId = "59a37e3f-471f-4958-a519-aa9ee128f5f7"

# Make a request to get all project items in the specified folder and filter out only ComputationalModel items
models: List[Any] = jinko.makeRequest(f'/app/v1/project-item?folderId={folderId}').json()
models = [m for m in models if m['type'] == "ComputationalModel"] 

# Create dictionaries to map model names to their corresponding SID and core ID
sid_dict = {m['name']: m['sid'] for m in models}
model_dict = {m['name']: m['coreId']['id'] for m in models}

# Query all labeled versions of a computational model
def getVersions(modelName, sid_dict):
    sid = sid_dict[modelName]
    versions: List[Any] = jinko.makeRequest(f'/app/v1/project-item/{sid}/versions?onlyLabeled=true').json()
    return versions

# Print the labeled versions available for this model
print( "All labeled model versions available for Test-Model:" )
print(json.dumps(getVersions('Test-Model', sid_dict), indent=4))

In the following cell, after taking care of formatting in the two helper functions *diffFormatCreator* and *formatKey*, we set up the function *reviewVersions* that will:
- download specified versions of the model (in the readable format outputted by the model editor, closely corresponding to the view on jinko)
- generate an HTML report of the differences of these two model versions using the packages difflib and deepdiff, with a section for each type of model component (parameters, species, reactions, ...)

The output folder for the downloaded models and the generated HTML report is an input to this function.

In [ ]:
# Function to format the diff output
def diffFormatCreator(m, a, b):
    result = ""
    delete = "<span style='color:red;'>"
    end = "</span>"
    insert = "<span style='color:green;'>"
    
    # Loop through the opcodes and format the changes
    for tag, i1, i2, j1, j2 in m.get_opcodes():
        if tag == 'replace':
            result += f' {delete}{" ".join(a[i1:i2])}{end} '
            result += f'{insert}{" ".join(b[j1:j2])}{end} '
        if tag == 'delete':
            result += delete
            result += ' '.join(a[i1:i2])
            result += end
        if tag == 'insert':
            result += insert
            result += ' '.join(b[j1:j2])
            result += end
        if tag == 'equal':
            result += ' '.join(a[i1:i2])
    return result

# Function to extract and format the diff keys
def formatKey(input_str, componentsHere):
    # Remove 'root' and split by the brackets
    parts = re.split(r"root|\[|\]|'|specifics|contents", input_str)

    # Get the component ID based on the index
    immutableIdHere = parts[3]
    idHere = [component for component in componentsHere if component['immutableId']==immutableIdHere][0]['id']
    
    # Filter out empty strings and join the remaining parts with spaces
    formatted_str = idHere + ' ' + ' '.join(filter(None, parts[4:]))

    return formatted_str

# Function to review two versions of a model by inputting their labels (default compares the last two labeled versions), 
# saving html reports in the path specified in outputPath
def reviewVersions(modelName, modelId, outputPath, newVersion='', oldVersion=''):
    # Query all labeled model versions for the specified model
    versions = getVersions(modelName, sid_dict)

    # Filter versions of interest
    if (newVersion!='' and oldVersion != ''):
        version_new = [ version for version in versions if version['label']==newVersion ][0]
        version_old = [ version for version in versions if version['label']==oldVersion ][0]
    elif (newVersion == '' and oldVersion == ''):
            version_new = versions[0]
            version_old = versions[1]
    else:
        print("Both newVersion and oldVersion need to be defined if a version is to be specified")
    
    # Extract snapshot IDs and labels
    snapshotId_new = version_new['coreId']['snapshotId']
    snapshotId_old = version_old['coreId']['snapshotId']
    label_new = version_new['label']
    label_old = version_old['label']
    
    # Download the model versions
    model_new = jinko.makeRequest(f'/core/v2/model_editor/jinko_model/{modelId}/snapshots/{snapshotId_new}').json()
    model_old = jinko.makeRequest(f'/core/v2/model_editor/jinko_model/{modelId}/snapshots/{snapshotId_old}').json()
    
    # Save the model versions as JSON files
    with open(f'{outputPath}/{modelName}-{label_new}.json', 'w') as f:
        json.dump(model_new, f, indent=4)
    with open(f'{outputPath}/{modelName}-{label_old}.json', 'w') as f:
        json.dump(model_old, f, indent=4)

    # Generate tags for the HTML diff
    tagsHere = ('<br>'.join([f'{tag["id"]}: {tag["immutableId"]}' for tag in model_new['tags']]))
    html_diff = f'<h1>Comparing new version {label_new} and old version {label_old} of {modelName}</h1>'
    html_diff += "Tags:<br>" + tagsHere + "<br><br>"
    
    # Compare components and generate HTML diff for each key
    for key in model_new['components'].keys():
        oldComponents = model_old['components'][key]
        newComponents = model_new['components'][key]
        for i in oldComponents:
            del i['diagnostics']
        for i in newComponents:
            del i['diagnostics']      
        # Use DeepDiff to find differences excluding diagnostics
        diffDict = DeepDiff(oldComponents, newComponents, exclude_regex_paths="root\[\d+\]\['diagnostics'\]", ignore_order=True, group_by='immutableId')
        # Check if there are any differences
        if diffDict != {}:
            html_diff += f'<h2>Changes in {key} </h2>'
            # Handle changed values
            if 'values_changed' in diffDict.keys(): 
                for idHere in diffDict['values_changed']:
                    old_value = str(diffDict['values_changed'][idHere]['old_value'])
                    new_value = str(diffDict['values_changed'][idHere]['new_value'])
                    m = difflib.SequenceMatcher(a=old_value.split(), b=new_value.split())
                    changes = diffFormatCreator(m, old_value.split(), new_value.split())
                    html_diff += f'Changes in {formatKey(idHere, oldComponents)}:<br>{changes}<br><br>'
                    
            # Handle added items
            if 'dictionary_item_added' in diffDict.keys(): 
                for idHere in diffDict['dictionary_item_added']:
                    clean_id = re.sub(r"root|\[|\]|'", "",idHere)
                    new_value = [component for component in newComponents if component['immutableId']==clean_id][0]
                    json_new_value = str(json.dumps(new_value, indent=4))
                    escaped_json_str = json_new_value.replace('\n', '<br>').replace(' ', '&nbsp;')
                    html_output = f"<span style='color:green;'>{escaped_json_str}</span>"
                    html_diff += f'Added {formatKey(idHere, newComponents)}:<br>{html_output}<br><br>'
                    
            # Handle removed items
            if 'dictionary_item_removed' in diffDict.keys(): 
                for idHere in diffDict['dictionary_item_removed']:
                    clean_id = re.sub(r"root|\[|\]|'", "",idHere)
                    old_value = [component for component in oldComponents if component['immutableId']==clean_id][0]
                    json_old_value = str(json.dumps(old_value, indent=4))
                    escaped_json_str = json_old_value.replace('\n', '<br>').replace(' ', '&nbsp;')
                    html_output = f"<span style='color:red;'>{escaped_json_str}</span>"
                    html_diff += f'Removed {formatKey(idHere, oldComponents)}:<br>{html_output}<br><br>'
            
        # Save the HTML diff
        with open(f'{outputPath}/{modelName}-{label_old}-{label_new}-diff.html', 'w') as f:
            f.write(html_diff)
    display(HTML(html_diff))

The latter function *reviewVersions* can now be used to generate HTML reports of the differences between two specified versions.

In [ ]:
# Example usage
# Local folder path to store downloaded model versions and HTML reports
outputPath = "outputs"
# Default review using the last two labeled versions
reviewVersions('Test-Model', model_dict['Test-Model'], outputPath)

#Reviews for specified versions
reviewVersions('Test-Model', model_dict['Test-Model'], outputPath, newVersion = "SeventhVersion", oldVersion = "SecondVersion")
reviewVersions('Test-Model', model_dict['Test-Model'], outputPath, newVersion = "FourthVersion", oldVersion = "ThirdVersion")
reviewVersions('Test-Model', model_dict['Test-Model'], outputPath, newVersion = "SecondVersion", oldVersion = "FirstVersion")